In [55]:
import os.path, csv
from pprint import pprint
from math import copysign, sqrt
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual

simulation_names = [name for name in os.listdir("gatling-results") if os.path.isfile(os.path.join("gatling-results", name, "simulation.log"))]

In [66]:
def init_bucket():
    return dict(
                active_user_count = 0,
                count = 0,
                mean = 0,
                M2 = 0.,
                min = float("+inf"),
                max = float("-inf"),
                median_avg = 0.,
                median = 0.,
            )

def update_bucket(bucket, sample):
    #Update extremes
    bucket['min'] = min(bucket['min'], sample)
    bucket['max'] = max(bucket['max'], sample)

    #Update mean/stdev (Welford's Algorithm)
    count, mean, M2 = bucket['count'], bucket['mean'], bucket['M2']
    count += 1
    delta = sample - mean
    mean += delta / count
    delta2 = sample - mean
    M2 += delta * delta2
    bucket['M2'] = M2
    bucket['count'] = count
    bucket['mean'] = mean

    #Update median (Jeff McClintock estimate)
    bucket['median_avg'] += (sample - bucket['median_avg']) * 0.1
    bucket['median'] += copysign( bucket['median_avg'] * 0.01, sample - bucket['median'] );

def finalize_bucket(bucket):
    count, mean, M2 = bucket['count'], bucket['mean'], bucket['M2']
    del bucket['M2']
    del bucket['median_avg']
    if count > 1:
        (mean, variance, sampleVariance) = (mean, M2/count, M2/(count - 1)) 
        bucket['stdev'] = sqrt(M2/count)
    else:
        bucket['stdev'] = float("nan")

def calculate_stats(name):
    print ("Processing log file for test run {}".format(name))
    user_add_timestamps = set()
    time_buckets = {}
    total_stats = dict(
                total = init_bucket(),
                fail = init_bucket(),
                success = init_bucket()
            )
    
    def process_user(scenario, userid, action, start_ts, end_ts):
        if action != "START":
            raise RuntimeError("Don't know what to do with USER line with action {}".format(action))
        if start_ts != end_ts:
            raise RuntimeError("Don't know how to handle USER START messages with start/end timestamps {} / {}.".format(start_ts, end_ts))
        user_add_timestamps.add(int(start_ts)//1000)
    
    def process_request(user, _1, request, start_ts, end_ts, result, _2):
        start_ts = float(start_ts)
        end_ts = float(end_ts)
        dur = end_ts - start_ts
        bucket_index = int(end_ts//1000)
        success = result == "OK"
        if bucket_index not in time_buckets:
            time_buckets[bucket_index] = dict(
                total = init_bucket(),
                fail = init_bucket(),
                success = init_bucket()
            )
        #bucket = time_buckets[bucket_index]
        update_bucket(total_stats['total'], dur)
        update_bucket(time_buckets[bucket_index]['total'], dur)
        if success:
            update_bucket(time_buckets[bucket_index]['success'], dur)
            update_bucket(total_stats['success'], dur)
        else:
            update_bucket(time_buckets[bucket_index]['fail'], dur)
            update_bucket(total_stats['fail'], dur)
        
    
    with open(os.path.join("gatling-results", name, "simulation.log")) as inf:
        for line in inf:
            line = line.rstrip("\n")
            type, *args = line.split("\t")
            if type == "RUN":
                continue
            elif type == "USER":
                process_user(*args)
            elif type == "REQUEST":
                if len(args) != 7:
                    print("Don't know how to handle REQUEST log {}".format(args))
                else:
                    process_request(*args)
            else:
                print(type, args)
        
    for add_ts in sorted(user_add_timestamps):
        total_stats['total']['active_user_count'] += 1
        total_stats['fail']['active_user_count'] += 1
        total_stats['success']['active_user_count'] += 1
        for bucket_ts in sorted(time_buckets):
            if bucket_ts >= add_ts:
                time_buckets[bucket_ts]['total']['active_user_count'] += 1
                time_buckets[bucket_ts]['fail']['active_user_count'] += 1
                time_buckets[bucket_ts]['success']['active_user_count'] += 1
        
    finalize_bucket(total_stats['total'])
    finalize_bucket(total_stats['fail'])
    finalize_bucket(total_stats['success'])
    for bucket_ts in sorted(time_buckets):
        finalize_bucket(time_buckets[bucket_ts]['total'])
        finalize_bucket(time_buckets[bucket_ts]['fail'])
        finalize_bucket(time_buckets[bucket_ts]['success'])
        
    return total_stats, time_buckets
        
    
simulations = {}
for name in simulation_names:
    stats_total, stats_by_time = calculate_stats(name)
    simulations[name] = dict(
        stats_total = stats_total,
        stats_by_time = stats_by_time,
        start_idx = sorted(stats_by_time)[0]
    )

Processing log file for test run primefactorkata-20191108073252673
Don't know how to handle REQUEST log ['46', '', '']


In [67]:
@interact
def plot_stats_over_time(name = simulation_names, field = simulations[simulation_names[0]]['stats_total']['total'].keys()):
    sim = simulations[name]
    stats = sim['stats_by_time']

    xs = np.arange(len(stats.keys()))
    fail_y = np.zeros(len(stats.keys()))
    success_y = np.zeros(len(stats.keys())) 
    total_y = np.zeros(len(stats.keys()))
    
    for idx, ts in enumerate(sorted(stats)):
        fail_y[idx] = stats[ts]['fail'][field]
        total_y[idx] = stats[ts]['total'][field]
        success_y[idx] = stats[ts]['success'][field]
        
    plt.rcParams["figure.figsize"] = (20,10)
    plt.plot(xs, total_y, label="Total")
    plt.plot(xs, success_y, label="OK")
    plt.plot(xs, fail_y, label="Failed")
    plt.legend()
    plt.show()

interactive(children=(Dropdown(description='name', options=('primefactorkata-20191108073252673',), value='prim…

In [68]:
@interact
def plot_comparison(field = simulations[simulation_names[0]]['stats_total']['total'].keys()):
    names = []
    values = []
    for name in simulation_names:
        v = simulations[name]['stats_total']['total'][field]
        names.append("{}\n{}".format(name,v))
        values.append(v)
        

    x = np.arange(len(names))
    plt.bar(x, values)
    plt.xticks(x, names)
    plt.show()

interactive(children=(Dropdown(description='field', options=('active_user_count', 'count', 'mean', 'min', 'max…